In [ ]:
#Dependencies
import pandas as pd
import requests
import json
import gmaps

#API key
from config import gkey
gmaps.configure(api_key=gkey)
#print(gkey)

In [ ]:
#Get list of cities from WeatherPy
city_df = pd.read_csv('weather_data_0428.csv')

print(city_df.shape)
print(city_df.columns)
city_df.head()

In [ ]:
## Generate heatmap of all cities, displaying humidity
locations = city_df[['Lat', 'Lng']]
weights = city_df['Humidity (%)']

fig1 = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=weights, max_intensity = 100, point_radius = 10)
fig1.add_layer(heatmap)
fig1

In [ ]:
## Narrow down data to find only places with ideal conditions

In [ ]:
#Condition 1: Temperature must be greater than 70°F, and less than 85°F
df_con1 = city_df.loc[city_df['Current Temp (F)'] > 70]
df_con1 = df_con1.loc[df_con1['Current Temp (F)'] < 85]

print(df_con1.shape)
df_con1.head()

In [ ]:
#Condition 2: Humidity must be less than 75%
df_con2 = df_con1.loc[df_con1['Humidity (%)'] < 75]

print(df_con2.shape)
df_con2.head()

In [ ]:
#Condition 3: Wind speed must be less than 12 mph
df_con3 = df_con2[df_con2['Wind Speed (mph)'] < 12]

print(df_con3.shape)
df_con3.head()

In [ ]:
#Restructuring dataframe
df_con3.index = range(len(df_con3.index))
vaca_df = df_con3[['City', 'Country', 'Lat', 'Lng', 'Current Temp (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)']]
vaca_df.head()

In [ ]:
## Pass data through Google Places API, find hotels in each city

In [ ]:
#Setting up url for requests

#Base url for place search
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

#Setting parameters for place search
rad = 5000
typ = 'lodging'

params = {'radius':rad, 'type':typ, 'key':gkey}

#Coordinates
lat = list(vaca_df['Lat'])
lng = list(vaca_df['Lng'])

In [ ]:
#Empty lists for iteration
hotel_name = []
hotel_lat = []
hotel_lng = []

#Generating API responses (takes 2 minutes+)
for l in range(len(lat)):
    params['location'] = str(lat[l]) + ',' + str(lng[l])
    response = requests.get(url, params=params)
    data = response.json()
    
    hotel_name.append(data['results'][0]['name'])
    hotel_lat.append(data['results'][0]['geometry']['location']['lat'])
    hotel_lng.append(data['results'][0]['geometry']['location']['lng'])
    
#hotel_name
#hotel_lat
#hotel_lng

In [ ]:
#Adding hotels to heatmap

#hotels_dict = {'name': hotel_name, 'lat': hotel_lat, 'lng': hotel_lng, 'city': list(vaca_df['City']), 'country': list(vaca_df['Country'])}
hotels_df = pd.DataFrame({'name': hotel_name, 'lat': hotel_lat, 'lng': hotel_lng, 'city': list(vaca_df['City']), 'country': list(vaca_df['Country'])})
hotels_df.tail()

In [ ]:
hotel_locations = hotels_df[['lat', 'lng']]
hotel_info = hotels_df.to_dict(orient='dict')

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_content = []

for h in range(len(hotel_name)):
    
    hname = hotel_info['name'][h]
    hcity = hotel_info['city'][h]
    hcoun = hotel_info['country'][h]
    
    hotel_content.append(f'<dl><dt>Hotel Name</dt><dd>{hname}</dd><dt>City</dt><dd>{hcity}</dd><dt>Country</dt><dd>{hcoun}</dd></dl>')
    
len(hotel_content)

In [ ]:
hotels_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_content)
fig2 = gmaps.figure()
fig2.add_layer(heatmap)
fig2.add_layer(hotels_layer)
fig2